# Week 10: March 13 to March 17
**Nicolas Escobar**

## Bayesian SEM

### Fixing BRMS issues

As we mentioned in last week's memo, we have implemented a simulation of a model that includes measurement error and latent variables. Here's a diagram of the model:

![thing](model.png)

Our hope is to implement non-parametric dependencies between some of the variables, but for now everything is linear. Last week, we tried fitting this synthethic data with BRMS, but we were getting a lot of divergences. Here's the basic syntax we're using:

In [ ]:
bf2 <- bf(Sm|mi() ~ mi(Xm) + Z1 + Z2)
bf3 <- bf(Y1 ~ 0 + mi(Sm))
bf4 <- bf(Y2 ~ 0 + mi(Sm))
bf5 <- bf(Y3 ~ 0 + mi(Sm))
bf8 <- bf(Xm|mi() ~ Z1 + Z2)
bf9 <- bf(W1 ~ 0 + mi(Xm))
bf10 <- bf(W2 ~ 0 + mi(Xm))
bf11 <- bf(W3 ~ 0 + mi(Xm))
bf12 <- bf(W4 ~ 0 + mi(Xm))

This week, we were able to get rid of those divergences by implementing more informative priors. Here's the full list of priors, in BRMS syntax:

In [ ]:
priors <- c(
    prior(normal(-1, 1), class = b, coef = Z1, resp = Sm),
    prior(normal(3, 1), class = b, coef = Z2, resp = Sm),
    prior(normal(2, 1), class = b, coef = miXm, resp = Sm),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y1),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y2),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y3),
    prior(normal(-1, 1), class = b, coef = Z1, resp = Xm),
    prior(normal(1, 1), class = b, coef = Z2, resp = Xm),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W1),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W2),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W3),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W4)
)

Unfortunately, some the estimated values for the parameters do not match the real ones yet. And we can see from the fit diagnostics that the effective sample size for them is too low. So we believe this will be fixed by running the chains for longer periods of time. 

### Mediation Analysis

Once the fit of the model is improved, the next step is to implement mediation analysis. We began taking steps in that direction. 

First, we tried using the `mediation` function from the `bayestestR` package. Despite trying all sorts of syntax combinations, it did not work. The problem seems to be that this package is unable to handle the latent factors. 

So, we implemented a small function that takes posterior draws from the model and computes the direct, indirect and total effects, as well as the mediation proportion. 

## Literature Review

We are starting to think about literature on the topic of Bayesian quantile regression on time to event of censored data. 

The primary reference is [Reich, Smith](https://pubmed.ncbi.nlm.nih.gov/23844559/). However, we're not satisfied with their exposition. 

We also went through [Zhang, Huang](https://pubmed.ncbi.nlm.nih.gov/31140028/). Unfortunately, although they do consider censored data, the quantile regression is not done on the time to event variable. 

We found three more references that we will look at. 